#### Initial data exploration and prep for `main.py`

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
load_dotenv()
# CTA_API_KEY = os.getenv('CTA_API_KEY')

cta_api_key = os.getenv('CTA_API_KEY')
postgres_password= os.getenv('POSTGRES_PASSWORD')


PDF link to CTA API documentation: https://www.transitchicago.com/assets/1/6/cta_Train_Tracker_API_Developer_Guide_and_Documentation.pdf

* will use the _locations API_ to track multiple locations at once
* produces a list of in service trains w basic info
* 193 [double check] total CTA stops maybe one dict per line
    * line_name = {cta_train_stop_key_id: friendltrain_name}
* json response most important
    * ctatt: root element
    * tmst: time stamp
    * train: container element one per train in response
    * destNm: friendly destination description
    * nextStpId: next stop ID matcing GTFS
    * nextStaId: next station ID matching GTFS
    * nextStaNm: proper name of next station
* do another response request for massive list of parent stop IDs {statId:StatNm}
    * parent stop IDs are for the station overall
    * each direction at the station has it's own ID for additional detail not needed for this project
* create another dictionary that maps StatId to gio pin
* link to total number of stations per train line: https://rtams.org/services/cta/lines
* {parent_station_id: GPIO_PIN}

In [ ]:
cta_api_key = os.getenv('CTA_API_KEY')

rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]


location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key=',
                            cta_api_key, 
                            '&rt=', rt_names[0],
                            '&rt=', rt_names[1],
                            '&rt=', rt_names[2],
                            '&rt=', rt_names[3],
                            '&outputType=JSON'])


# location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
#                                 ,cta_api_key
#                                 ,'&rt=', rt_names[-4]
#                                 ,'&rt=', rt_names[-3]
#                                 ,'&rt=', rt_names[-2]
#                                 ,'&rt=', rt_names[-1]
#                                 ,'&outputType=JSON'])


response = requests.get(location_url).json()

print(response)

locations_df = pd.DataFrame()

df_cols = ['train_line_name',
               'train_direction_id',
               'next_station_parent_id',
               'next_stop_id',
               'arrival_time',
               'prediction_time',
               'is_app',
               'is_delayed' ]

stop = 0
train = 0

all_train_line_name = []
all_direction_id = []
all_nxt_sta_p_id = []
all_nxt_stop_id = []
all_arrival_times = []
all_pred_times = []
all_is_app = []
all_is_dly = []


# list of lists to build df
all_lapi_trn_data = []
all_nxt_sta_dict= {}

rt_len = len(response['ctatt']['route'])
for train in range(rt_len):
        for stop in range(len(response['ctatt']['route'][train]['train'])):
                all_nxt_sta_dict[response['ctatt']['route'][train]['train'][stop]['nextStaId']] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                 # repeat this line a lot maybe build funtion for it
                # all_nxt_sta_names.append(response['ctatt']['route'][train]['train'][stop]['nextStaNm'])
                # def append_data(append_list: list, append_col: col) -> list
                all_train_line_name.append(response['ctatt']['route'][train]['@name'])
                all_direction_id.append(response['ctatt']['route'][train]['train'][stop]['trDr'])
                all_nxt_sta_p_id.append(response['ctatt']['route'][train]['train'][stop]['nextStaId'])
                all_nxt_stop_id.append(response['ctatt']['route'][train]['train'][stop]['nextStpId'])
                all_arrival_times.append(response['ctatt']['route'][train]['train'][stop]['arrT'])
                all_pred_times.append(response['ctatt']['route'][train]['train'][stop]['prdt'])
                all_is_app.append(response['ctatt']['route'][train]['train'][stop]['isApp'])
                all_is_dly.append(response['ctatt']['route'][train]['train'][stop]['isDly'])

                stop+=1
        train+=1

all_lapi_trn_data.append(all_train_line_name)
all_lapi_trn_data.append(all_direction_id)

all_lapi_trn_data.append(all_nxt_sta_p_id)
all_lapi_trn_data.append(all_nxt_stop_id )
all_lapi_trn_data.append(all_arrival_times)
all_lapi_trn_data.append(all_pred_times)
all_lapi_trn_data.append(all_is_app)
all_lapi_trn_data.append(all_is_dly)

lapi_df = pd.DataFrame(dict(zip(df_cols, all_lapi_trn_data)))

print(all_nxt_sta_dict)

{'ctatt': {'tmst': '2026-02-27T23:23:57', 'errCd': '0', 'errNm': None, 'route': [{'@name': 'red', 'train': [{'rn': '834', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40910', 'nextStpId': '30177', 'nextStaNm': '63rd', 'prdt': '2026-02-27T23:23:37', 'arrT': '2026-02-27T23:24:37', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.77498', 'lon': '-87.6271', 'heading': '321'}, {'rn': '843', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40100', 'nextStpId': '30020', 'nextStaNm': 'Morse', 'prdt': '2026-02-27T23:23:13', 'arrT': '2026-02-27T23:25:13', 'isApp': '0', 'isDly': '0', 'flags': None, 'lat': '42.00836', 'lon': '-87.66591', 'heading': '358'}, {'rn': '844', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40340', 'nextStpId': '30066', 'nextStaNm': 'Berwyn', 'prdt': '2026-02-27T23:23:40', 'arrT': '2026-02-27T23:24:40', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.97345', 'lon': '-87.65853', 'heading': '358'}, {'rn': '

### building and joining dfs

think about how to incorporate `@name` into df since stop ids don't break out to that detail
    * do additional research on dupe stations in csv
    * efficient way to add the same `@name` for one chunk of foor loop while adding the right number to df
* am i gonna have to build out my own final stop id table that breaks down color_stopname_dir
    * LED lights direction won't matter just pink or green at ashland
    * but for db and building out accurate model could be interesting data to have...

In [ ]:
lapi_df.head()

,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,40910,30177,2026-02-27T23:24:37,2026-02-27T23:23:37,1,0
1,40100,30020,2026-02-27T23:25:13,2026-02-27T23:23:13,0,0
2,40340,30066,2026-02-27T23:24:40,2026-02-27T23:23:40,1,0
3,40630,30121,2026-02-27T23:25:31,2026-02-27T23:23:31,0,0
4,41000,30193,2026-02-27T23:24:30,2026-02-27T23:23:30,1,0


In [ ]:
# first 0 is train second 0 is stop
print(response['ctatt']['route'][0]['train'][0])

{'rn': '717', 'destSt': '30182', 'destNm': 'Midway', 'trDr': '5', 'nextStaId': '40930', 'nextStpId': '30182', 'nextStaNm': 'Midway', 'prdt': '2026-02-26T21:57:57', 'arrT': '2026-02-26T21:58:57', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.79788', 'lon': '-87.72957', 'heading': '248'}


In [111]:
# data downloaded from 
# https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme/about_data

metadata_df = pd.read_csv('CTA_List_of_\'L\'_Stops_20260227.csv')
# metadata_df[metadata_df['STOP_ID'] == 30182]
metadata_df[metadata_df['STATION_DESCRIPTIVE_NAME'].str.contains("Ashland")]


,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Y,Pnk,O,Location
38,30033,W,Ashland (Harlem-54th/Cermak-bound),Ashland,Ashland (Green & Pink lines),40170,True,False,False,True,False,False,False,True,False,"(41.885269, -87.666969)"
39,30032,E,Ashland (Loop-63rd-bound),Ashland,Ashland (Green & Pink lines),40170,True,False,False,True,False,False,False,True,False,"(41.885269, -87.666969)"
40,30206,S,Ashland (Midway-bound),Ashland,Ashland (Orange Line),41060,True,False,False,False,False,False,False,False,True,"(41.839234, -87.665317)"
41,30205,N,Ashland (Loop-bound),Ashland,Ashland (Orange Line),41060,True,False,False,False,False,False,False,False,True,"(41.839234, -87.665317)"
42,30057,W,Ashland/63rd (Terminal arrival),Ashland/63rd,Ashland/63rd (Green Line),40290,True,False,False,True,False,False,False,False,False,"(41.77886, -87.663766)"
43,30056,E,Ashland/63rd (Harlem-bound),Ashland/63rd,Ashland/63rd (Green Line),40290,True,False,False,True,False,False,False,False,False,"(41.77886, -87.663766)"


In [18]:
lapi_df['next_stop_id'] = lapi_df['next_stop_id'].astype(int)

joined_df = lapi_df.merge(metadata_df[['STOP_ID', 'STOP_NAME', 'STATION_DESCRIPTIVE_NAME', 'DIRECTION_ID']],
                          left_on='next_stop_id', right_on='STOP_ID', how='left', suffixes=["_fact", "_dim"]).head()

cols =['next_stop_id', 'STOP_NAME',
       'next_station_parent_id',
       'STATION_DESCRIPTIVE_NAME',
       'arrival_time', 'prediction_time',
       'is_app', 'is_delayed',
       'DIRECTION_ID']

joined_df[cols]

,next_stop_id,STOP_NAME,next_station_parent_id,STATION_DESCRIPTIVE_NAME,arrival_time,prediction_time,is_app,is_delayed,DIRECTION_ID
0,30177,63rd (Howard-bound),40910,63rd (Red Line),2026-02-27T23:24:37,2026-02-27T23:23:37,1,0,N
1,30020,Morse (Howard-bound),40100,Morse (Red Line),2026-02-27T23:25:13,2026-02-27T23:23:13,0,0,N
2,30066,Berwyn (Howard-bound),40340,Berwyn (Red Line),2026-02-27T23:24:40,2026-02-27T23:23:40,1,0,N
3,30121,Clark/Division (Howard-bound),40630,Clark/Division (Red Line),2026-02-27T23:25:31,2026-02-27T23:23:31,0,0,N
4,30193,Cermak-Chinatown (Howard-bound),41000,Cermak-Chinatown (Red Line),2026-02-27T23:24:30,2026-02-27T23:23:30,1,0,N


### postgres connection + write df to table

In [34]:
postgres_password= os.getenv('POSTGRES_PASSWORD')

password = postgres_password
dbname = 'cta_data'

postgres_url = ''.join(['postgresql://postgres:',password, '@localhost:5432/',dbname])
engine = create_engine(postgres_url)

metadata_df.to_sql(name="src_stop_metadata",
        con=engine,
        schema='raw',
        if_exists='replace',
        index=False)
# print(postgres_url)

302

In [ ]:
def write_df_to_pgdb(df: pd.DataFrame, table: str, if_exists: str, has_index: bool):

    postgres_password= os.getenv('POSTGRES_PASSWORD')

    password = postgres_password
    dbname = 'cta_data'

    print('connecting to postgres db...')

    postgres_url = ''.join(['postgresql://postgres:',password, '@localhost:5432/',dbname])
    engine = create_engine(postgres_url)

    df.to_sql(name=table,
        con=engine,
        schema='raw',
        if_exists=if_exists,
        index=has_index)
        
    engine.dispose()

    print(f'wrote {df.shape} rows, columns to {table}')


### lapi  notes

Goal:
* set up local postgres database to store train tracker info
* two dataframes
    * one that will have _all_ station ids, station names, and useful metadata
    * second for storing info from the api
    * key of dictionary is col name value is array that will populate column
        * `dict(zip(my_list_1, my_list_2))`

        * next parent station id
            * parent stop id [non directional]
            * `response['ctatt']['route'][0]['train'][0]['nextStaId']`
        * stop id
            * not parent id
            * ex 30219 
            * join on stop id since stop id is unique and parent id is not
            * `response['ctatt']['route'][0]['train'][0]['nextStpId']`
        * station arrival time
            * prediction of train arrival time
            * `response['ctatt']['route'][0]['train'][0]['arrT']`
            * ex. 2026-02-26T19:35:19
        * station prediction time
            * timestamp for when _prediction_ was generated
            * `response['ctatt']['route'][0]['train'][0]['prdt']`
            * ex. 2026-02-26T19:33:19
        * is station delayed
            * bool to denote if tain is delayed or not
            * `response['ctatt']['route'][0]['train'][0]['isDly']`
        * is app
            * bool to denote app showing "approaching" "due"
            `response['ctatt']['route'][0]['train'][0]['isApp']`
---
additional col locations 
* station direction ID, TrDr
    * numeric code that translates to direction
    * ex. for red line 1 is howard bound 5 is 95th bound
    * codes will only be 1 **or** 5
    * `response['ctatt']['route'][0]['train'][0]['trDr']`
* cardinal direction
    * N, S, E, W
        * `response['ctatt']['route'][0]['train'][0]['heading']`
* train line name
    * have each train line map to an ID and store ID to train @name in seperate db table?
    * `response['ctatt']['route'][0]['@name']`



        * next station name
            * ex. Kedzie
            * `response['ctatt']['route'][0]['train'][0]['nextStaNm']`
        * after df is built and clean dtypes load it into local postgres db
            * `pd.to_sql("table_name", if_exists=append, index=False, con=con)`
            * `conn = psycopg2.connect(
                dbname="school",
                user="postgres",
                password="your_password",
                host="localhost")`
                * https://www.geeksforgeeks.org/postgresql/postgresql-connecting-to-the-database-using-python/
---

### DBT Notes

1. load tables from raw schema into dev schema
    * create new schema when loading
2. basic cleaning of stg tables final filtering w fct/dim tables

    **high level**
    * rename columns w AS
    * clean up dtypes w CAST
    * generate surrogate keys
    * correctly map primary and foreign keys in schema .yml file
    * format datetime to `9999-12-31 23:59:59` format is currently `2026-03-01T12:35:27` should it be in UTC?
        * use REPLACE(string, old_value, new_value)
---
**details**
* direction table
    * generate surrogate primary key from `train_line_name` and `train_direction_id`
    * drop index after surrogate primary key is created
* lapi table
    * drop test row and generate new surrogate primary key?
    * replace `train_direction_id` and `train_line_name` with new surrogate primary key from direction table
        * use case when statment
    * clean up arrival and predicion time
        * REPLACE(`2026-03-01T12:35:27`, `T`, ` `)
        * cast time columns as datetime
* stop table
    * lowercase all column names
    * map stop_id as primary key
        * stop_ids are unique to know train line join to direction table
    * rename `direction_id` to `cardinal_direction_id`
        
3. load cleaned tables into final fact/dim tables to final schema
---
* need the main functions
    1. rename + lowercase table columns
        * rename w `AS` keyword
        * update data types w `CAST`
            * https://github.com/kirthanavivekanadan/dbt-schema-rename-demo/blob/main/models/staging/orders/stg_orders.sql
    2. combine columns to map to unique primary key
        * generate a surrogate key with `{{ dbt_utils.generate_surrogate_key('direction_id', 'train_line_name') }}`
            * https://popsql.com/learn-dbt/dbt-utils-surrogate-keys#dbt_utils-for-surrogate-key-generation
        * update columns w case when statements
            * https://docs.getdbt.com/sql-reference/case
    3. correctly map foreign and primary keys
        * defined in schema .yml files
    4. reference dbt tables
        * `with directions as (
    select * from {{ ref('stg_direction') }})`
    5. grab initial tables from raw
        * https://github.com/kirthanavivekanadan/dbt-schema-rename-demo/blob/main/models/staging/customers/stg_customer.sql
    6. grabbing data from source
        * https://github.com/kirthanavivekanadan/dbt-schema-rename-demo/blob/main/models/src_raw.yml

---
**Getting started w dbt**
* https://publish.obsidian.md/datavidhya/Course+Notes/Fundamentals+of+Data+Engineering
* install `dbt-core` and `dbt-postgres` with uv
* create directory using `mkdir ~/.dbt`
* initialize dbt project w dbt init `my_dbt_project_dir_name`
* seperate schema for raw and staging, fact, and dim models
    * raw for raw tables dev for staging, fact, and dim
    * follow set up steps and for threads keep 1
1. step 1 create initial models for tables in `models/staging`
    * after creating first model fo `dbt run` must run from folder that containers `dbt_project.yml`
        * to run one model `dbt run --model <my_model_name>` [omit.sql file extension]
    * edit `dbt_project.yml` to change default materialization for models
        * materialize all staging models as views and dim/fact as tables
        * rename columns and cast here
2. step 2 preform more serious transofrmations from staging tables
    * create dim/fact models
        * materialzie as tables
        * `models/dim` and `models/fct`
        * edit `~/.dbt/profiles.yml` schema to write to a new schema for dbt
    * generate surrogate direction_id primary key
    * clean and update timestamps for lapi
    * case when for lapi for direction_id
    * filtering w where or order
    * `{{ ref('file_name.sql') }}` file name from `models/`

* can create `models/sources.yml` file to define useful source information like database, schema, table names, primary key, foreign key, column tests
        


### data model

Database >> schema >> tables
* schemas: raw & fact/dim >> _star schema_
    * raw: for csv dim table and raw load of lapi fact table
        * for initial load of models [tables]
        * materiazlie as view
    * staging: for dbt transformations
    * fact/dim: output of  transformations
        * materiazlie as table
        * dev schema?
        


1. train_locations: fact table from lapi
    * cols: `['unique_primary_key', 'train_line_name,
           'train_direction_id', 'next_station_parent_id',
           'next_stop_id', 'arrival_time',
           'prediction_time',
           'is_app', 'is_delayed' ]`
    * **primary key**: unique primary key when data is written to table
    * **foreign keys:** `next_stop_id`, new `direction_id` col
    * **data transformations:**
        * create primary key --> drop test value fix primary key serial?
        * ~~`next_station_parent_id` & `next_stop_id` as int~~
        * ~~`arrival_time` & `prediction_time` read as time stamps~~
        * ~~`is_app` & `is_delayed` as bool~~
        * format arrival and prediction time timestamp
        * add tests of dtypes to cols
        * replace train_line_name and train_direction_id w unique direction id from direction table?

2. stop: dimension table from csv
    * cols: `['STOP_ID', DIRECTION_ID', 'STOP_NAME', 'STATION_NAME', 'STATION_DESCRIPTIVE_NAME',
       'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Y', 'Pnk', 'O',
       'Location']`
    * **primary key:** `STOP_ID`
    * **foreign keys:** none
    * **data transformations:** rename `stop_id` or `next_stop_id` so col names match?,
                            lower case col names,
                            map stop id as primary key
                            rename direction_id to cardinal_direction_id?

3. directions:dimension from lapi documentation
    * cols: = [index, train_line_name, train_direction_id, train_direction_name]
    * **primary key:** create new unique id from direction id and train name named `direction_id`
    * **foreign keys:** none
    * **data transformations:** rename index to id and set as primary key

* https://www.tecmint.com/install-postgresql-with-pgadmin4-on-linux-mint/
---

### final functions

In [ ]:
def get_half_trns_lapi_call(is_first_half_rt_nms: bool) -> pd.DataFrame:

    cta_api_key = os.getenv('CTA_API_KEY')

    rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]

    
    if is_first_half_rt_nms==1: 
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,cta_api_key
                                ,'&rt=', rt_names[0]
                                ,'&rt=', rt_names[1]
                                ,'&rt=', rt_names[2]    # api has max of four train lines at once
                                ,'&rt=', rt_names[3]
                                ,'&outputType=JSON'])

    else:
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,cta_api_key
                                ,'&rt=', rt_names[-4]
                                ,'&rt=', rt_names[-3]
                                ,'&rt=', rt_names[-2]
                                ,'&rt=', rt_names[-1]
                                ,'&outputType=JSON'])
        

    response = requests.get(location_url).json()


    stop = 0
    train = 0
    rt_len = len(response['ctatt']['route'])

    all_train_line_name = []
    all_train_direction_id = []
    all_nxt_sta_p_id = []
    all_nxt_stop_id = []
    all_arrival_times = []
    all_pred_times = []
    all_is_app = []
    all_is_dly = []


    # list of lists to build df
    all_lapi_trn_data = []
    lapi_df = pd.DataFrame()
    df_cols = ['train_line_name',
               'train_direction_id',
               'next_station_parent_id',
               'next_stop_id',
               'arrival_time',
               'prediction_time',
               'is_app',
               'is_delayed' ]


    for train in range(rt_len):
        
        for stop in range(len(response['ctatt']['route'][train]['train'])): # active stops on train line
                 # repeat this line a lot maybe build funtion for it
                # def append_data(append_list: list, append_col: col) -> list
                all_train_line_name.append(response['ctatt']['route'][train]['@name'])
                all_train_direction_id.append(response['ctatt']['route'][train]['train'][stop]['trDr'])
                all_nxt_sta_p_id.append(response['ctatt']['route'][train]['train'][stop]['nextStaId'])
                all_nxt_stop_id.append(response['ctatt']['route'][train]['train'][stop]['nextStpId'])
                all_arrival_times.append(response['ctatt']['route'][train]['train'][stop]['arrT'])
                all_pred_times.append(response['ctatt']['route'][train]['train'][stop]['prdt'])
                all_is_app.append(response['ctatt']['route'][train]['train'][stop]['isApp'])
                all_is_dly.append(response['ctatt']['route'][train]['train'][stop]['isDly'])
                
                stop+=1

        train+=1

    all_lapi_trn_data.append(all_train_line_name)
    all_lapi_trn_data.append(all_train_direction_id)
    all_lapi_trn_data.append(all_nxt_sta_p_id)
    all_lapi_trn_data.append(all_nxt_stop_id )
    all_lapi_trn_data.append(all_arrival_times)
    all_lapi_trn_data.append(all_pred_times)
    all_lapi_trn_data.append(all_is_app)
    all_lapi_trn_data.append(all_is_dly)

    lapi_df = pd.DataFrame(dict(zip(df_cols, all_lapi_trn_data)))

        
    # return all_nxt_sta_dict
    return lapi_df

In [5]:
first_half_df = get_half_trns_lapi_call(is_first_half_rt_nms=True)

In [18]:
first_half_df.head()

,train_line_name,train_direction_id,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,red,1,41320,30255,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0
1,red,1,40240,30046,2026-03-01T12:34:58,2026-03-01T12:33:58,1,0
2,red,5,40910,30178,2026-03-01T12:35:12,2026-03-01T12:33:12,0,0
3,red,5,41000,30194,2026-03-01T12:36:03,2026-03-01T12:33:03,0,0
4,red,5,40650,30126,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0


In [ ]:
first_half_copy_df = first_half_df.copy()

first_half_copy_df['id'] = None

first_half_copy_df = first_half_copy_df[['id',
                    'train_line_name',
                    'train_direction_id',
                    'next_station_parent_id',
                    'next_stop_id',
                    'arrival_time',
                    'prediction_time',
                    'is_app',
                    'is_delayed']].head()

,id,train_line_name,train_direction_id,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,None,red,1,41320,30255,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0
1,None,red,1,40240,30046,2026-03-01T12:34:58,2026-03-01T12:33:58,1,0
2,None,red,5,40910,30178,2026-03-01T12:35:12,2026-03-01T12:33:12,0,0
3,None,red,5,41000,30194,2026-03-01T12:36:03,2026-03-01T12:33:03,0,0
4,None,red,5,40650,30126,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0


In [26]:
first_half_copy_df.head()

,id,train_line_name,train_direction_id,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,None,red,1,41320,30255,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0
1,None,red,1,40240,30046,2026-03-01T12:34:58,2026-03-01T12:33:58,1,0
2,None,red,5,40910,30178,2026-03-01T12:35:12,2026-03-01T12:33:12,0,0
3,None,red,5,41000,30194,2026-03-01T12:36:03,2026-03-01T12:33:03,0,0
4,None,red,5,40650,30126,2026-03-01T12:35:27,2026-03-01T12:33:27,0,0


In [103]:
first_half_df.to_sql(name='src_lapi_data', con=engine, schema='raw',
                     if_exists='replace')



47

In [29]:
write_df_to_pgdb(df=first_half_df, table='src_lapi_data',
                 if_exists='append', has_index=False) 

connecting to postgres db...
wrote (57, 8) rows, columns to src_lapi_data


In [14]:
second_half_df = get_half_trns_lapi_call(is_first_half_rt_nms=False)

In [30]:
# second_half_df[second_half_df['next_stop_id'] == '40510']
second_half_df.head()

,train_line_name,train_direction_id,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,org,5,41400,30081,2026-03-01T12:52:03,2026-03-01T12:49:03,0,0
1,org,1,41400,30080,2026-03-01T12:50:01,2026-03-01T12:49:01,1,0
2,org,1,40120,30022,2026-03-01T12:49:57,2026-03-01T12:48:57,1,0
3,org,5,40930,30182,2026-03-01T12:49:39,2026-03-01T12:48:39,1,0
4,org,5,41130,30216,2026-03-01T12:51:54,2026-03-01T12:48:54,0,0


In [107]:
# first_half_df[first_half_df['next_station_parent_id'] == '40510']
first_half_df.shape

(47, 6)

In [31]:
write_df_to_pgdb(second_half_df, table='src_lapi_data',
                 if_exists='append', has_index=False)

connecting to postgres db...
wrote (15, 8) rows, columns to src_lapi_data


#### Creating direction ids df / table

In [ ]:
train_directions_df = pd.DataFrame()

train_directions_df_cols = [
    'train_line_name',
    'train_direction_id',
    'train_direction_name']

train_directions_data = [
    ['blue', 'blue', 'y', 'y','brn', 'brn',
    'p', 'p','pink', 'pink','g', 'g',
    'red', 'red', 'org', 'org'], 
    [1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5],
    ['O’Hare-bound', 'Forest Park-bound',
     'Skokie-bound', 'Howard-bound',
     'Kimball-bound', 'Loop-bound',
     'Linden-bound', 'Howard- or Loop-bound',
     'Loop-bound', '54th/Cermak-bound',
     'Harlem/Lake-bound', 'Ashland/63rd- or Cottage Grove-bound',
     'Howard-bound', '95th/Dan Ryan-bound',
     'Loop-bound', 'Midway-bound']]

train_directions_df = pd.DataFrame(dict(zip(train_directions_df_cols, train_directions_data)),
                                   index=[i for i in range(1, 17)])

train_directions_df[train_directions_df['train_line_name'] == 'y']

train_directions_df.head()

# print([i for i in range(1, 17)])
# dict()

,train_line_name,train_direction_id,train_direction_name
1,blue,1,O’Hare-bound
2,blue,5,Forest Park-bound
3,y,1,Skokie-bound
4,y,5,Howard-bound
5,brn,1,Kimball-bound


In [66]:
write_df_to_pgdb(df=train_directions_df, table='src_direction_metadata',
                 if_exists='replace', has_index=True)

connecting to postgres db...
wrote (16, 3) rows, columns to src_direction_metadata


* need to do another API call or just download file [explore + research pros and cons] to get all CTA station stops to map to all GPIO pins
* probably only do call once when running map and store data but continue to run `get_half_trns_lapi_call` to get up to date train location info